##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Cómo migrar LoggingTensorHook y StopAtStepHook a las retrollamadas de Keras

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/migrate/logging_stop_hook">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/logging_stop_hook.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/guide/migrate/logging_stop_hook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar  bloc de notas</a></td>
</table>

En TensorFlow 1, se utiliza `tf.estimator.LoggingTensorHook` para monitorizar y registrar tensores, mientras que `tf.estimator.StopAtStepHook` ayuda a detener el entrenamiento en un paso específico cuando se entrena con `tf.estimator.Estimator`. Este bloc de notas muestra cómo migrar de estas API a sus equivalentes en TensorFlow 2 mediante retrollamadas personalizadas de Keras (`tf.keras.callbacks.Callback`) con `Model.fit`.

Las [retrollamadas](https://www.tensorflow.org/guide/keras/custom_callback) de Keras son objetos que se llaman en diferentes puntos durante el entrenamiento/evaluación/predicción en las API incorporadas de Keras `Model.fit`/`Model.evaluate`/`Model.predict`. Puede obtener más información sobre las retrollamadas en la documentación de la API `tf.keras.callbacks.Callback`, así como en las guías [Escribiendo sus propias retrollamadas](../..guide/keras/custom_callback.ipynb/) y [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (la sección *Usando retrollamadas*). Para migrar de `SessionRunHook` a TensorFlow 1 mediante las retrollamadas de Keras en TensorFlow 2, consulte la guía [del entrenamiento para migrar con lógica asistida](sessionrunhook_callback.ipynb).

## Preparación

Empiece con imports y un conjunto de datos sencillo a modo de demostración:

In [ ]:
import tensorflow as tf
import tensorflow.compat.v1 as tf1

In [ ]:
features = [[1., 1.5], [2., 2.5], [3., 3.5]]
labels = [[0.3], [0.5], [0.7]]

# Define an input function.
def _input_fn():
  return tf1.data.Dataset.from_tensor_slices((features, labels)).batch(1)

## TensorFlow 1: Registro de tensores y parada del entrenamiento con las API de tf.estimator

En TensorFlow 1, se definen varios hooks para controlar el comportamiento del entrenamiento. Posteriormente, se pasan estos hooks a `tf.estimator.EstimatorSpec`.

En el siguiente ejemplo:

- Para monitorear o registrar tensores, por ejemplo, pesos o pérdidas del modelo, se utiliza `tf.estimator.LoggingTensorHook` (`tf.train.LoggingTensorHook` es su alias).
- Para detener el entrenamiento en un paso específico, se utiliza `tf.estimator.StopAtStepHook` (`tf.train.StopAtStepHook` es su alias).

In [ ]:
def _model_fn(features, labels, mode):
  dense = tf1.layers.Dense(1)
  logits = dense(features)
  loss = tf1.losses.mean_squared_error(labels=labels, predictions=logits)
  optimizer = tf1.train.AdagradOptimizer(0.05)
  train_op = optimizer.minimize(loss, global_step=tf1.train.get_global_step())

  # Define the stop hook.
  stop_hook = tf1.train.StopAtStepHook(num_steps=2)

  # Access tensors to be logged by names.
  kernel_name = tf.identity(dense.weights[0])
  bias_name = tf.identity(dense.weights[1])
  logging_weight_hook = tf1.train.LoggingTensorHook(
      tensors=[kernel_name, bias_name],
      every_n_iter=1)
  # Log the training loss by the tensor object.
  logging_loss_hook = tf1.train.LoggingTensorHook(
      {'loss from LoggingTensorHook': loss},
      every_n_secs=3)

  # Pass all hooks to `EstimatorSpec`.
  return tf1.estimator.EstimatorSpec(mode,
                                     loss=loss,
                                     train_op=train_op,
                                     training_hooks=[stop_hook,
                                                     logging_weight_hook,
                                                     logging_loss_hook])

estimator = tf1.estimator.Estimator(model_fn=_model_fn)

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
estimator.train(_input_fn)

## TensorFlow 2: Registrar tensores y detener el entrenamiento con retrollamadas personalizadas y Model.fit

En TensorFlow 2, cuando se utiliza Keras `Model.fit` (o `Model.evaluate`) para el entrenamiento/evaluación, se puede configurar la monitorización del tensor y la parada del entrenamiento definiendo Keras `tf.keras.callbacks.Callback`s personalizado. Luego, los puede transferir al parámetro `callbacks` de `Model.fit` (o `Model.evaluate`). (Obtenga más información en la guía [Escribiendo sus propias retrollamadas](../..guide/keras/custom_callback.ipynb)).

En el siguiente ejemplo:

- Para recrear las funcionalidades de `StopAtStepHook`, defina una retrollamada personalizada (denominada `StopAtStepCallback`) en la que anule el método `on_batch_end` para parar el entrenamiento después de un determinado número de pasos.
- Para recrear el comportamiento de `LoggingTensorHook`, defina una retrollamada personalizada (`LoggingTensorCallback`) en la que registre y extraiga manualmente los tensores registrados, ya que no es posible acceder a los tensores por su nombre. También puede implementar la frecuencia de registro dentro de la retrollamada personalizada. En el siguiente ejemplo se imprimirán los pesos cada dos pasos. También son posibles otras estrategias como el registro cada N segundos.

In [ ]:
class StopAtStepCallback(tf.keras.callbacks.Callback):
  def __init__(self, stop_step=None):
    super().__init__()
    self._stop_step = stop_step

  def on_batch_end(self, batch, logs=None):
    if self.model.optimizer.iterations >= self._stop_step:
      self.model.stop_training = True
      print('\nstop training now')

class LoggingTensorCallback(tf.keras.callbacks.Callback):
  def __init__(self, every_n_iter):
      super().__init__()
      self._every_n_iter = every_n_iter
      self._log_count = every_n_iter

  def on_batch_end(self, batch, logs=None):
    if self._log_count > 0:
      self._log_count -= 1
      print("Logging Tensor Callback: dense/kernel:",
            model.layers[0].weights[0])
      print("Logging Tensor Callback: dense/bias:",
            model.layers[0].weights[1])
      print("Logging Tensor Callback loss:", logs["loss"])
    else:
      self._log_count -= self._every_n_iter

Cuando termine, transfiera las nuevas retrollamadas -`StopAtStepCallback` y `LoggingTensorCallback` al parámetro `callbacks` de `Model.fit`:

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((features, labels)).batch(1)
model = tf.keras.models.Sequential([tf.keras.layers.Dense(1)])
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.05)
model.compile(optimizer, "mse")

# Begin training.
# The training will stop after 2 steps, and the weights/loss will also be logged.
model.fit(dataset, callbacks=[StopAtStepCallback(stop_step=2),
                              LoggingTensorCallback(every_n_iter=2)])

## Siguientes pasos

Obtenga más información sobre las retrollamadas en:

- Documentos de la API: `tf.keras.callbacks.Callback`
- Guía: [Escribiendo sus propias retrollamadas](../..guide/keras/custom_callback.ipynb/)
- Guía: [Entrenamiento y evaluación con los métodos incorporados](https://www.tensorflow.org/guide/keras/train_and_evaluate) (la sección *Usar retrollamadas*)

También le pueden resultar útiles los siguientes recursos relacionados con la migración:

- La [Guía de migración de parada anticipada](early_stopping.ipynb): `tf.keras.callbacks.EarlyStopping` es una retrollamada de parada anticipada incorporada.
- La [Guía de migración de TensorBoard](tensorboard.ipynb): TensorBoard permite el seguimiento y la visualización de métricas
- La [guía del entrenamiento para migrar a la lógica asistida](sessionrunhook_callback.ipynb): De `SessionRunHook` en TensorFlow 1 a las retrollamadas de Keras en TensorFlow 2